In [ ]:
# !pip install selenium
# !pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import bs4
import urllib
import urllib.request
import time
from bs4 import BeautifulSoup
import lxml
import html5lib
import requests
import warnings
import json

In [4]:
warnings.filterwarnings('ignore')

In [5]:
chrome_options = Options()
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(executable_path='chromedriver.exe', options=chrome_options)

In [6]:
url = f'https://www.copart.com/vehicleFinder/'
driver.get(url)

In [7]:
li_css = driver.find_elements_by_css_selector('li')
list_li = [li.text for li in li_css if '(' in li.text]
list_li = [x.replace(' ','').replace(')','').replace('/','').replace('-','').lower() for x in list_li]
d = {}
for li in list_li:
    d[f"{li.split('(')[0]}"] = f"{li.split('(')[-1]}"
    
a_css = driver.find_elements_by_css_selector('a')
a_links = [x.get_attribute('href') for x in a_css if '/search/' in str(x.get_attribute('href'))]

dict_links = {}
for link in a_links:
    key = link.split('search/')[-1].split('?')[0]
    dict_links[key] = link
    
dict_L = {}
dict_L['feat_items'] = ['classics', 'featuredvehicles', 'hotitems', 'nolicenserequired', 'puresaleitems', 'runanddrive', 'exotics', 'hybridvehicles', 'offsitesales', 'sealedbidrepossessions', 'rentals', 'buyitnow', 'newitems', 'fleetlease', 'recoveredthefts', 'copartgo']
dict_L['vehi_types'] = ['atvs', 'boats', 'forklifts', 'industrialequipment', 'mediumdutyboxtrucks', 'snowmobiles', 'trailers', 'dirtbikes', 'jetskis', 'recreationalvehicles', 'automobiles', 'motorcycles', 'heavydutytrucks']
dict_L['pop_brands'] = ['audi', 'cadillac', 'chrysler', 'ford', 'honda', 'infiniti', 'kia', 'mazda', 'mitsubishi', 'pontiac', 'suzuki', 'volkswagen', 'bmw', 'dodge', 'hyundai', 'lexus', 'nissan', 'toyota', 'gmc', 'mercedesbenz', 'jeep', 'subaru', 'chevrolet', 'volvo', 'harley', 'landrover', 'acura', 'buick', 'freightliner', 'international', 'jaguar', 'lincoln', 'mini', 'polaris', 'porsche', 'ram', 'yamaha', 'hummer', 'isuzu', 'lamborghini', 'lotus', 'maserati', 'rollsroyce', 'saab', 'scion']
dict_L['bids_based'] = ['nobidsyet', 'lotswithbids']
dict_L['damage_typ'] = ['biohazardouschemical', 'burnengine', 'burn', 'allover', 'burninterior', 'damagehistory', 'framedamage', 
    'frontend', 
    'mechanical', 'normalwear', 'rearend', 'side', 'toproof', 
    'vandalism', 
    'haildamage', 'partialincompleterepair', 'stripped', 'waterflood', 'rollover', 'undercarriage', 'minordentsscratches']
dict_L['owner_type'] = ['nonrepairable', 'salvagetitle', 'cleantitle']
dict_L['source_lnk'] = ['impoundvehicles', 'repossessions', 'donations']

In [8]:
for name_new, name_old in zip([x for x in dict_links if x not in d],[x for x in d if x not in dict_links]):
    d[name_new] = d[name_old]

In [ ]:
# frontend se tomaron range(10) # Son como 900
# vandalism se tomaron range(10) # Son como 21
# mechanical se tomaron range(20) # Son como 49

In [9]:
pprint

Pretty printing has been turned OFF


In [10]:
dict_L['damage_typ']

['biohazardouschemical', 'burnengine', 'burn', 'allover', 'burninterior', 'damagehistory', 'framedamage', 'frontend', 'mechanical', 'normalwear', 'rearend', 'side', 'toproof', 'vandalism', 'haildamage', 'partialincompleterepair', 'stripped', 'waterflood', 'rollover', 'undercarriage', 'minordentsscratches']

In [11]:
['biohazardouschemical', 'burnengine', 'burn', 'allover', 'burninterior', 'damagehistory', 'framedamage', 'frontend', 'mechanical', 'normalwear', 'rearend', 'side', 'toproof', 'vandalism', 'haildamage', 'partialincompleterepair', 'stripped', 'waterflood', 'rollover', 'undercarriage', 'minordentsscratches']

['biohazardouschemical', 'burnengine', 'burn', 'allover', 'burninterior', 'damagehistory', 'framedamage', 'frontend', 'mechanical', 'normalwear', 'rearend', 'side', 'toproof', 'vandalism', 'haildamage', 'partialincompleterepair', 'stripped', 'waterflood', 'rollover', 'undercarriage', 'minordentsscratches']

In [ ]:
# falta frontend, rearend, side

In [73]:
damage_type = dict_L['damage_typ'][11]
print(damage_type)
n_pages = int(d[damage_type]) // 100 + 4
print(d[damage_type])
print(n_pages)

side
22455
228


In [74]:
### DEFINE DAMAGE TYPE TO LOOK FOR
driver.get(dict_links[damage_type])
time.sleep(5)

n_entries = driver.find_element_by_xpath('//*[@id="serverSideDataTable_length"]/label/select')
n_100 = driver.find_element_by_xpath('//*[@id="serverSideDataTable_length"]/label/select/option[3]')

n_entries.click()
n_100.click()


In [61]:
n_pages = 74

In [ ]:
### GET THE CAR LINKS AND INITIAL FEATURES
lot_autos = {}
for i in range(n_pages):
    html = driver.find_element_by_tag_name('html')
    time.sleep(1)
    for i in range(230):
        html.send_keys(Keys.DOWN)
    html.send_keys(Keys.HOME)
    time.sleep(1)
    try:
        for row in range(1,110):
            features_car = {}
            try: 
                link_lt = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[3]/div/a')
                print(link_lt.text)
                features_car['damage_type'] = damage_type
                features_car['link'] = link_lt.get_attribute('href')
                features_car['link_view_all'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[2]/div[2]/a').get_attribute('href')
                features_car['small_img_car'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[2]/div[1]/a/img').get_attribute('src')
                if (features_car['small_img_car'] == '') or (features_car['small_img_car']==None):
                    features_car['small_img_car'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[2]/div[1]/a/img').get_attribute('src')
                features_car['year'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[4]').text
                features_car['make'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[5]').text
                features_car['model'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[6]').text
                features_car['item_n'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[7]').text
                features_car['location'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[8]').text
                features_car['saledate'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[9]').text
                features_car['odometer'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[10]').text
                features_car['doc_type'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[11]').text
                features_car['damage'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[12]').text
                features_car['est_retail_val'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[13]').text
                features_car['currentbid'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[14]').text
            except:
                continue
            lot_autos[link_lt.text] = features_car

            with open(f'{damage_type}.json', 'w') as fp:
                json.dump(lot_autos, fp)

        next_button = driver.find_element_by_xpath('//*[@id="serverSideDataTable_next"]/a')
        next_button.click()
        time.sleep(0.5)
    except:
        continue

50918380
25823190
26318580
28107970
32282170
33544330
33567759
35903370
36401570
38048880
38056680
38339000
38617370
38876670
39076510
40087310
40122930
40503310
40641410
41340459
41436850
41488100
41637020
41742040
41974730
42043990
42192010
42218690
42220820
42843110
42849250
43024500
43113960
43237880
43454550
43457570
43476950
43605950
43610950
43611760
43617480
43676620
43740210
43765640
43804880
44325110
44755390
44755510
44881690
45046230
45150430
45425440
45436660
45436830
45475310
45484830
45543460
45548330
45562120
45571390
45581080
45710710
45743450
45794400
45835340
45872380
45903900
46007170
46036540
46201800
46219190
46235500
46314690
46381070
46598440
46607970
46648290
46744130
46747240
46820400
46823820
46901360
46930900
46981300
47008610
47009960
47052260
47107360
47111080
47135340
47160810
47192789
47199930
47217700
47305770
47343150
47358240
47422810
47464530
47567260
47597890
47610660
47611990
47662890
47690920
47757890
47806870
47824740
47824870
47838770
47888840
4

In [68]:
### GET THE CAR LINKS AND INITIAL FEATURES
n_entries = driver.find_element_by_xpath('//*[@id="serverSideDataTable_length"]/label/select')
n_100 = driver.find_element_by_xpath('//*[@id="serverSideDataTable_length"]/label/select/option[3]')

n_entries.click()
n_100.click()

time.sleep(1)

n_pages = int(d[damage_type]) // 100 + 1

lot_autos = {}
for i in range(n_pages):
    html = driver.find_element_by_tag_name('html')
    for i in range(200):
        html.send_keys(Keys.DOWN)
#         time.sleep(0.3)
    html.send_keys(Keys.HOME)
    try:
        for row in range(1,110):
            features_car = {}
            try: 
                link_lt = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[3]/div/a')
                print(link_lt.text)
                features_car['damage_type'] = damage_type
                features_car['link'] = link_lt.get_attribute('href')
                features_car['link_view_all'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[2]/div[2]/a').get_attribute('href')
                features_car['small_img_car'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[2]/div[1]/a/img').get_attribute('src')
                if (features_car['small_img_car'] == '') or (features_car['small_img_car']==None):
                    features_car['small_img_car'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[2]/div[1]/a/img').get_attribute('src')
                features_car['year'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[4]').text
                features_car['make'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[5]').text
                features_car['model'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[6]').text
                features_car['item_n'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[7]').text
                features_car['location'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[8]').text
                features_car['saledate'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[9]').text
                features_car['odometer'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[10]').text
                features_car['doc_type'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[11]').text
                features_car['damage'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[12]').text
                features_car['est_retail_val'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[13]').text
                features_car['currentbid'] = driver.find_element_by_xpath(f'//*[@id="serverSideDataTable"]/tbody/tr[{row}]/td[14]').text
            except:
                continue
            lot_autos[link_lt.text] = features_car
            
            with open(f'{damage_type}.json', 'w') as fp:
                json.dump(lot_autos, fp)

        next_button = driver.find_element_by_xpath('//*[@id="serverSideDataTable_next"]/a')
        next_button.click()
        time.sleep(0.5)
    except:
        continue

46181720
46181720
39228210
39228210
40236760
40236760
42298420
42298420
42527580
42527580
44413400
44413400
45697460
45697460
46070910
46070910
47412860
47412860
47847920
47847920
47926280
47926280
49163930
49163930
49468990
49468990
49802210
49802210
49953190
49953190
50618200
50618200
51342020
51342020
51501540
51501540
51659020
51659020
51779050
51779050
52440740
52440740
52491390
52491390
52749630
52749630
53030930
53030930
53195510
53195510
53296750
53296750
53376720
53376720
53522750
53522750
53581350
53581350
53889920
53889920
53935170
53935170
53956390
53956390
54111520
54111520
55098130
55098130
55497610
55497610
56583380
56583380
33925089
33925089
37583150
37583150
38121350
38121350
40993290
40993290
43643990
43643990
46700690
46700690
46898280
46898280
47534600
47534600
49202410
49202410
49226950
49226950
49409890
49409890
50056500
50056500
51420400
51420400
51618480
51618480
52136010
52136010
52152940
52152940
53011420
53011420
53193000
53193000
53288290
53288290
54145240
5

In [ ]:
### DEFINE DAMAGE TYPE TO LOOK FOR
damage_type = 'rearend'
driver.get(dict_links[damage_type])
time.sleep(2)

In [ ]:
### GET THE CAR ADVANCED FEATURES AND IMAGES
print(len(lot_autos))
i=0
for car_code in lot_autos:
    print(i)
    i+=1
    try:        
        driver.get(lot_autos[car_code]['link'])
        
        features_car = lot_autos[car_code]

        time.sleep(1)

        view_all = driver.find_element_by_class_name('viewalltxt')
        view_all.click()
        time.sleep(1)
        
        img_links = []
        for img_id in range(1,16):
            try:
                xpath_image = f'//*[@id="lot-images-wrapper"]/div/div[{img_id}]/div[1]/img'
                image = driver.find_element_by_xpath(xpath_image)
                img_links.append(image.get_attribute('src'))
            except: continue
        
        features_car['img_links'] = img_links

        features_car['car_title'] = driver.find_element_by_css_selector('h1').text
        for row_features in range(1,50):
            try:
                xpath_feature = f'//*[@id="lot-details-bottom"]/div[1]/div/div/div[1]/div[1]/div/div[{row_features}]'
                texto = driver.find_element_by_xpath(xpath_feature).text
                features_car[texto.split('\n')[0]] = texto.split('\n')[-1]
            except: continue
        for row_bid in range(1,50):
            try:
                xpath_bid_feat = f'//*[@id="lot-details-bottom"]/div[2]/div[1]/div/div[2]/div/div/div/div/div/div/div/div/div/div[2]/div[{row_bid}]'
                texto = driver.find_element_by_xpath(xpath_bid_feat).text
                features_car[texto.split('\n')[0]] = texto.split('\n')[-1]
            except: continue
        for row_sale in range(1,50):
            try:
                xpath_bid_feat = f'//*[@id="lot-details-bottom"]/div[2]/div[3]/div[2]/div[{row_sale}]'
                texto = driver.find_element_by_xpath(xpath_bid_feat).text
                features_car[texto.split('\n')[0]] = texto.split('\n')[-1]
            except: continue
        try:
            del features_car['']
        except: pass
    
        lot_autos[car_code] = features_car
        
        with open(f'{damage_type}.json', 'w') as fp:
            json.dump(lot_autos, fp)
    except:
        pass

2000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [182]:
features_car

{'Brand': '2014 TOYOTA RAV4 LE',
 'VIN:': '2T3BFREV2EW******',
 'Doc Type:': 'NJ - CERT OF TITLE-SALVAGE',
 'Odometer:': '129,518 mi (NOT ACTUAL)',
 'Highlights:': 'Run and Drive',
 'Primary Damage:': 'BIOHAZARD/CHEMICAL',
 'Secondary Damage:': 'REAR END',
 'Est. Retail Value:': '$11,484 USD',
 'Body Style:': '4DR SPOR',
 'Vehicle Type:': 'AUTOMOBILE',
 'Color:': 'SILVER',
 'Engine Type:': '2.5L 4',
 'Cylinders:': '4',
 'Transmission:': 'AUTOMATIC',
 'Drive:': 'All wheel drive',
 'Fuel:': 'GAS',
 'Keys:': 'YES',
 'Notes:': 'There are no Notes for this Lot',
 'Bid Status:': "YOU HAVEN'T BID",
 'Eligibility Status': 'Check Now',
 'Sale Status:': 'On Approval',
 'Starting Bid:': '$5,600 USD',
 'All bids are binding and all sales are final.': 'What This Means',
 'Location:': 'NJ - SOMERVILLE',
 'Sale Date:': 'Add to Calendar',
 'Sale Name:': 'NJ - SOMERVILLE',
 'Lane/Item/Grid/Row:': 'B/2039/A060',
 'Time Left:': '0D 0H 40min',
 'Last Updated:': '11/19/2020 4:14 pm'}